In [1]:
import pandas as pd
import numpy as np

In [5]:
accidents = pd.read_csv('../data/2015-2019_dataset_clusters.csv')

In [13]:
accidents.head()

,x,y,accident_id,year,address,severity,accident_type,severity_numeric,borough_geo,timestamp,date,hour,month_name,month,day,cluster_id
0,-74.105296,4.509792,4437952,2016,CL 80A-KR 1 SE 02,Injury,Crash,8,USME,2016-02-27 16:20:00+00:00,2016-02-27,16,Feb,2,Sat,0
1,-74.167225,4.631051,4472304,2017,AV AVENIDA CIUDAD DE CALI-CL 42 S 02,Injury,Run over,9,KENNEDY,2017-02-09 16:45:00+00:00,2017-02-09,16,Feb,2,Thu,1
2,-74.121790,4.603106,4512837,2018,AV AVENIDA PRIMERA DE MAYO-KR 50A 14,Injury,Crash,8,PUENTE ARANDA,2018-03-25 12:10:00+00:00,2018-03-25,12,Mar,3,Sun,1
3,-74.075332,4.607944,4437462,2016,KR 13-CL 18 02,Injury,Crash,8,SANTA FE,2016-02-22 08:55:00+00:00,2016-02-22,8,Feb,2,Mon,1
4,-74.064965,4.744182,4473374,2017,AV AVENIDA BOYACA-CL 160 02,Injury,Run over,9,SUBA,2017-02-19 13:00:00+00:00,2017-02-19,13,Feb,2,Sun,2


In [14]:
df_positive_2019 = accidents[pd.to_datetime(accidents.date).dt.year == 2019]

In [15]:
df_positive = df_positive_2019[['x','y','date','hour', 'borough_geo', 'address', 'cluster_id']].copy()

In [16]:
df_positive.date = pd.to_datetime(df_positive.date)

In [17]:
df_positive['day_of_year'] = df_positive.date.dt.dayofyear

In [18]:
df_positive['coords'] = df_positive[['x', 'y']].apply(lambda x: ';'.join((str(x['x']), str(x['y']))), axis = 1)

In [19]:
df_positive = df_positive[['hour', 'day_of_year','borough_geo', 'coords', 'address', 'cluster_id']].copy()

In [25]:
df_positive.head(2)

,hour,day_of_year,borough_geo,coords,address,cluster_id
0,14,11,TUNJUELITO,-74.13436280399995;4.579297398000051,KR 26-CL 52B S 2,470
1,11,11,CHAPINERO,-74.04881864699999;4.665144043000055,CL 84-KR 8 2,1


In [24]:
df_positive = df_positive.reset_index(drop=True)

In [21]:
def generate_negative_sample(row):
    num_samples = 0
    while num_samples < 1:
        randcol = np.random.choice(np.array(['hour', 'day_of_year', 'coords']))
        replacement = df_positive.sample()
        negative_sample = row
        negative_sample[randcol] = replacement[randcol].values[0]
        if randcol == 'coords':
            negative_sample['borough_geo'] = replacement['borough_geo'].values[0]
            negative_sample['address']  = replacement['address'].values[0]
            negative_sample['cluster_id']  = replacement['cluster_id'].values[0]

        ncoords = negative_sample['coords']
        nhour = negative_sample['hour']
        nday = negative_sample['day_of_year']
        
        if df_positive.loc[(df_positive['coords'] == ncoords) & (df_positive['hour'] == nhour) & (df_positive['day_of_year'] == nday)].empty:
            yield negative_sample[['hour', 'day_of_year','coords','address', 'borough_geo', 'cluster_id']].values.tolist()
            num_samples += 1
        else:
            print('found coincidence', negative_sample[['hour', 'day_of_year','coords']].values.tolist())

In [26]:
my_list = [generate_negative_sample(df_positive.iloc[x]) for x in df_positive.index]

In [27]:
%%time
from itertools import chain
neg_list = list(chain.from_iterable(my_list))

607404199994;4.640868394000051']
found coincidence [11, 277, '-74.16537835399998;4.620097236000049']
found coincidence [18, 283, '-74.12740286199994;4.669526482000038']
found coincidence [10, 121, '-74.07854970199999;4.646602019000056']
found coincidence [12, 177, '-74.07260532799995;4.6945902490000435']
found coincidence [20, 176, '-74.10560590799997;4.648621565000042']
found coincidence [10, 303, '-74.08107437099994;4.595120953000047']
found coincidence [14, 187, '-74.12784229499994;4.629271811000024']
found coincidence [19, 89, '-74.07256651899995;4.731098954000061']
found coincidence [7, 117, '-74.04569900699995;4.726790397000059']
found coincidence [16, 71, '-74.07307526799997;4.598059717000068']
found coincidence [13, 9, '-74.09793445899999;4.713315476000048']
found coincidence [21, 115, '-74.04650097899997;4.7516098330000345']
found coincidence [5, 116, '-74.09517720799995;4.558947506000038']
found coincidence [18, 306, '-74.11426600399994;4.641167307000046']
found coincidence [

In [28]:
neg_df = pd.DataFrame.from_records(neg_list)

In [29]:
neg_df

,0,1,2,3,4,5
0,14,158,-74.13436280399995;4.579297398000051,KR 26-CL 52B S 2,TUNJUELITO,470
1,16,11,-74.04881864699999;4.665144043000055,CL 84-KR 8 2,CHAPINERO,1
2,23,11,-74.08542502399995;4.632912610000061,CL 26-KR 37 2,TEUSAQUILLO,1
3,7,11,-74.02506831099998;4.725288041000057,KR 7-CL 150 2,USAQUEN,146
4,7,12,-74.08077001499998;4.670083261000059,CL 68-KR 58 2,BARRIOS UNIDOS,1
...,...,...,...,...,...,...
29115,16,337,-74.13568662999995;4.580801026000074,KR 28-DG 52B S 2,TUNJUELITO,383
29116,4,12,-74.084274684;4.590370525000028,KR 10-CL 1B 02,SANTA FE,1
29117,16,277,-74.09775169999993;4.619584603000021,CL 13-KR 37 35,PUENTE ARANDA,1
29118,19,354,-74.17421686799997;4.576242357000068,KR 75-CL 75D S 02,CIUDAD BOLIVAR,-1


In [30]:
neg_df.columns = ['hour', 'day_of_year', 'coords', 'address', 'borough', 'cluster_id']

In [31]:
neg_df['year'] = 2019

In [32]:
neg_df.shape

(29120, 7)

In [33]:
def compose_date(years, months=1, days=1, weeks=None, hours=None, minutes=None,
                 seconds=None, milliseconds=None, microseconds=None, nanoseconds=None):
    years = np.asarray(years) - 1970
    months = np.asarray(months) - 1
    days = np.asarray(days) - 1
    types = ('<M8[Y]', '<m8[M]', '<m8[D]', '<m8[W]', '<m8[h]',
             '<m8[m]', '<m8[s]', '<m8[ms]', '<m8[us]', '<m8[ns]')
    vals = (years, months, days, weeks, hours, minutes, seconds,
            milliseconds, microseconds, nanoseconds)
    return sum(np.asarray(v, dtype=t) for t, v in zip(types, vals)
               if v is not None)


neg_df['timestamp'] = compose_date(neg_df['year'], days=neg_df['day_of_year'], hours=neg_df['hour'], minutes=np.zeros(neg_df.shape[0]), seconds=np.zeros(neg_df.shape[0]))
neg_df['date'] = compose_date(neg_df['year'], days=neg_df['day_of_year'])


In [34]:
neg_df = neg_df.join(neg_df.coords.str.split(';',expand=True))

In [35]:
neg_df.rename(columns={0: "x", 1: "y"}, inplace=True)

In [36]:
neg_df

,hour,day_of_year,coords,address,borough,cluster_id,year,timestamp,date,x,y
0,14,158,-74.13436280399995;4.579297398000051,KR 26-CL 52B S 2,TUNJUELITO,470,2019,2019-06-07 14:00:00,2019-06-07,-74.13436280399995,4.579297398000051
1,16,11,-74.04881864699999;4.665144043000055,CL 84-KR 8 2,CHAPINERO,1,2019,2019-01-11 16:00:00,2019-01-11,-74.04881864699999,4.665144043000055
2,23,11,-74.08542502399995;4.632912610000061,CL 26-KR 37 2,TEUSAQUILLO,1,2019,2019-01-11 23:00:00,2019-01-11,-74.08542502399995,4.632912610000061
3,7,11,-74.02506831099998;4.725288041000057,KR 7-CL 150 2,USAQUEN,146,2019,2019-01-11 07:00:00,2019-01-11,-74.02506831099998,4.725288041000057
4,7,12,-74.08077001499998;4.670083261000059,CL 68-KR 58 2,BARRIOS UNIDOS,1,2019,2019-01-12 07:00:00,2019-01-12,-74.08077001499998,4.670083261000059
...,...,...,...,...,...,...,...,...,...,...,...
29115,16,337,-74.13568662999995;4.580801026000074,KR 28-DG 52B S 2,TUNJUELITO,383,2019,2019-12-03 16:00:00,2019-12-03,-74.13568662999995,4.580801026000074
29116,4,12,-74.084274684;4.590370525000028,KR 10-CL 1B 02,SANTA FE,1,2019,2019-01-12 04:00:00,2019-01-12,-74.084274684,4.590370525000028
29117,16,277,-74.09775169999993;4.619584603000021,CL 13-KR 37 35,PUENTE ARANDA,1,2019,2019-10-04 16:00:00,2019-10-04,-74.09775169999993,4.619584603000021
29118,19,354,-74.17421686799997;4.576242357000068,KR 75-CL 75D S 02,CIUDAD BOLIVAR,-1,2019,2019-12-20 19:00:00,2019-12-20,-74.17421686799997,4.576242357000068


In [62]:
neg_df['accident_id'] = ''
neg_df['severity'] = ''
neg_df['accident_type'] = ''
neg_df['severity_numeric'] = ''
neg_df['day_of_week'] = neg_df.timestamp.dt.dayofweek
neg_df['day'] = neg_df.timestamp.dt.day
neg_df['month'] = neg_df.timestamp.dt.month

In [63]:
neg_df.head()

,hour,day_of_year,coords,address,borough,cluster_id,year,timestamp,date,x,y,accident_id,severity,accident_type,severity_numeric,day_of_week,month,day
0,14,158,-74.13436280399995;4.579297398000051,KR 26-CL 52B S 2,TUNJUELITO,470,2019,2019-06-07 14:00:00,2019-06-07,-74.13436280399995,4.579297398000051,,,,,4,6,7
1,16,11,-74.04881864699999;4.665144043000055,CL 84-KR 8 2,CHAPINERO,1,2019,2019-01-11 16:00:00,2019-01-11,-74.04881864699999,4.665144043000055,,,,,4,1,11
2,23,11,-74.08542502399995;4.632912610000061,CL 26-KR 37 2,TEUSAQUILLO,1,2019,2019-01-11 23:00:00,2019-01-11,-74.08542502399995,4.632912610000061,,,,,4,1,11
3,7,11,-74.02506831099998;4.725288041000057,KR 7-CL 150 2,USAQUEN,146,2019,2019-01-11 07:00:00,2019-01-11,-74.02506831099998,4.725288041000057,,,,,4,1,11
4,7,12,-74.08077001499998;4.670083261000059,CL 68-KR 58 2,BARRIOS UNIDOS,1,2019,2019-01-12 07:00:00,2019-01-12,-74.08077001499998,4.670083261000059,,,,,5,1,12


In [42]:
accidents.drop(columns=['accident_id', 'month_name'], inplace=True)

In [51]:
accidents_2019 = accidents[accidents.year == 2019]

In [52]:
accidents_2019.timestamp = pd.to_datetime(accidents_2019.timestamp, utc=True).dt.tz_localize(None)

In [55]:
accidents_2019['day'] = accidents_2019.timestamp.dt.day
accidents_2019['day_of_week'] = accidents_2019.timestamp.dt.dayofweek
accidents_2019['day_of_year'] = accidents_2019.timestamp.dt.dayofyear

In [57]:
accidents_2019.head(20)

,x,y,year,address,severity,accident_type,severity_numeric,borough_geo,timestamp,date,hour,month,day,cluster_id,day_of_year,day_of_week
106678,-74.134363,4.579297,2019,KR 26-CL 52B S 2,Injury,Crash,8,TUNJUELITO,2019-01-11 14:20:00,2019-01-11,14,1,11,470,11,4
106679,-74.048819,4.665144,2019,CL 84-KR 8 2,Material damage,Crash,1,CHAPINERO,2019-01-11 11:30:00,2019-01-11,11,1,11,1,11,4
106680,-74.173301,4.615589,2019,KR 80-CL 53A S 2,Injury,Crash,8,KENNEDY,2019-01-11 23:20:00,2019-01-11,23,1,11,1,11,4
106681,-74.025068,4.725288,2019,KR 7-CL 150 2,Material damage,Crash,1,USAQUEN,2019-01-11 06:30:00,2019-01-11,6,1,11,146,11,4
106682,-74.080770,4.670083,2019,CL 68-KR 58 2,Injury,Crash,8,BARRIOS UNIDOS,2019-01-10 07:20:00,2019-01-10,7,1,10,1,10,3
106683,-74.151311,4.572193,2019,KR 38-CL 61 S 2,Material damage,Crash,1,CIUDAD BOLIVAR,2019-01-11 09:20:00,2019-01-11,9,1,11,1,11,4
106684,-74.142010,4.667770,2019,CL 17-KR 96B 71,Material damage,Crash,1,FONTIBON,2019-01-11 01:00:00,2019-01-11,1,1,11,14,11,4
106685,-74.135317,4.599616,2019,KR 68-CL 38H S 02,Injury,Crash,8,KENNEDY,2019-01-09 23:48:00,2019-01-09,23,1,9,1,9,2
106686,-74.135804,4.588874,2019,CL 49A-KR 37 S 2,Injury,Crash,8,TUNJUELITO,2019-01-11 17:38:00,2019-01-11,17,1,11,-1,11,4
106687,-74.149876,4.687513,2019,CL 22F-KR 116 02,Injury,Crash,8,FONTIBON,2019-01-10 16:30:00,2019-01-10,16,1,10,274,10,3


In [58]:
accidents_2019 = accidents_2019[['x', 'y', 'timestamp', 'date', 'year', 'month',
       'day', 'day_of_year', 'day_of_week', 'hour', 'address', 'severity', 'accident_type',
       'severity_numeric', 'borough_geo','cluster_id']]

In [59]:
accidents_2019.rename(columns={"borough_geo": "borough"}, inplace=True)

In [60]:
accidents_2019['sample_type'] = 1

In [64]:
neg_df = neg_df[['x', 'y', 'timestamp', 'date', 'year', 'month',
       'day', 'day_of_year', 'day_of_week', 'hour', 'address', 'severity', 'accident_type',
       'severity_numeric', 'borough','cluster_id']]

In [65]:
neg_df['sample_type'] = 0

In [71]:
accidents_2019.shape

(29120, 17)

In [72]:
dataset = pd.concat([accidents_2019,neg_df])

In [73]:
dataset

,x,y,timestamp,date,year,month,day,day_of_year,day_of_week,hour,address,severity,accident_type,severity_numeric,borough,cluster_id,sample_type
106678,-74.1344,4.5793,2019-01-11 14:20:00,2019-01-11,2019,1,11,11,4,14,KR 26-CL 52B S 2,Injury,Crash,8,TUNJUELITO,470,1
106679,-74.0488,4.66514,2019-01-11 11:30:00,2019-01-11,2019,1,11,11,4,11,CL 84-KR 8 2,Material damage,Crash,1,CHAPINERO,1,1
106680,-74.1733,4.61559,2019-01-11 23:20:00,2019-01-11,2019,1,11,11,4,23,KR 80-CL 53A S 2,Injury,Crash,8,KENNEDY,1,1
106681,-74.0251,4.72529,2019-01-11 06:30:00,2019-01-11,2019,1,11,11,4,6,KR 7-CL 150 2,Material damage,Crash,1,USAQUEN,146,1
106682,-74.0808,4.67008,2019-01-10 07:20:00,2019-01-10,2019,1,10,10,3,7,CL 68-KR 58 2,Injury,Crash,8,BARRIOS UNIDOS,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29115,-74.13568662999995,4.580801026000074,2019-12-03 16:00:00,2019-12-03 00:00:00,2019,12,3,337,1,16,KR 28-DG 52B S 2,,,,TUNJUELITO,383,0
29116,-74.084274684,4.590370525000028,2019-01-12 04:00:00,2019-01-12 00:00:00,2019,1,12,12,5,4,KR 10-CL 1B 02,,,,SANTA FE,1,0
29117,-74.09775169999993,4.619584603000021,2019-10-04 16:00:00,2019-10-04 00:00:00,2019,10,4,277,4,16,CL 13-KR 37 35,,,,PUENTE ARANDA,1,0
29118,-74.17421686799997,4.576242357000068,2019-12-20 19:00:00,2019-12-20 00:00:00,2019,12,20,354,4,19,KR 75-CL 75D S 02,,,,CIUDAD BOLIVAR,-1,0


In [74]:
boroughs = pd.read_csv("../data/localidades.csv")

In [75]:
boroughs.borough.unique()

array(['USAQUEN', 'CHAPINERO', 'SANTA FE', 'SAN CRISTOBAL', 'USME',
       'TUNJUELITO', 'BOSA', 'KENNEDY', 'FONTIBON', 'ENGATIVA', 'SUBA',
       'BARRIOS UNIDOS', 'TEUSAQUILLO', 'LOS MARTIRES', 'ANTONIO NARIÑO',
       'PUENTE ARANDA', 'CANDELARIA', 'RAFAEL URIBE URIBE',
       'CIUDAD BOLIVAR', 'SUMAPAZ'], dtype=object)

In [76]:
boroughs.columns

Index(['id', 'borough', 'area_km2', 'population', 'population_density'], dtype='object')

In [6]:
# boroughs.replace({"Tunjuelito": 'TUNJUELITO', "Chapinero":'CHAPINERO', "Kennedy": 'KENNEDY', "Usaquén": 'USAQUEN', "BarriosUnidos":  'BARRIOS UNIDOS', "CiudadBolívar": 
# 'CIUDAD BOLIVAR', "Fontibón":  'FONTIBON', "PuenteAranda":  'PUENTE ARANDA', "Suba":  'SUBA', "Teusaquillo":
# 'TEUSAQUILLO', "SantaFe":  'SANTA FE', "Bosa":  'BOSA', "SanCristóbal":  'SAN CRISTOBAL', "Engativá":  'ENGATIVA', "RafaelUribeUribe": 
# 'RAFAEL URIBE URIBE', "AntonioNariño":  'ANTONIO NARIÑO', "LosMártires":  'LOS MARTIRES', "LaCandelaria":
# 'CANDELARIA', "Usme": 'USME', "Sumapaz":  'SUMAPAZ'}, inplace=True)

In [77]:
# boroughs.drop(columns = ['id'], inplace=True)

In [78]:
dataset = dataset.merge(boroughs, how='left', left_on='borough', right_on='borough')

In [79]:
dataset.head()

,x,y,timestamp,date,year,month,day,day_of_year,day_of_week,hour,...,severity,accident_type,severity_numeric,borough,cluster_id,sample_type,id,area_km2,population,population_density
0,-74.1344,4.5793,2019-01-11 14:20:00,2019-01-11,2019,1,11,11,4,14,...,Injury,Crash,8,TUNJUELITO,470,1,6,9.91,199430,20124.11
1,-74.0488,4.66514,2019-01-11 11:30:00,2019-01-11,2019,1,11,11,4,11,...,Material damage,Crash,1,CHAPINERO,1,1,2,38.15,139701,3661.88
2,-74.1733,4.61559,2019-01-11 23:20:00,2019-01-11,2019,1,11,11,4,23,...,Injury,Crash,8,KENNEDY,1,1,8,38.59,1088443,28205.31
3,-74.0251,4.72529,2019-01-11 06:30:00,2019-01-11,2019,1,11,11,4,6,...,Material damage,Crash,1,USAQUEN,146,1,1,65.31,501999,7686.40
4,-74.0808,4.67008,2019-01-10 07:20:00,2019-01-10,2019,1,10,10,3,7,...,Injury,Crash,8,BARRIOS UNIDOS,1,1,12,11.90,243465,20459.24


In [81]:
dataset.rename(columns={"id": "borough_id"}, inplace=True)

In [83]:
weather = pd.read_csv("../data/total.csv")

In [84]:
weather.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'time', 'summary', 'icon',
       'precipIntensity', 'precipProbability', 'temperature',
       'apparentTemperature', 'dewPoint', 'humidity', 'windSpeed',
       'cloudCover', 'uvIndex', 'visibility', 'windBearing', 'location',
       'windGust', 'pressure', 'precipType', 'ozone', 'precipAccumulation'],
      dtype='object')

In [85]:
weather.drop(columns=["Unnamed: 0", "Unnamed: 0.1"], inplace=True)

In [86]:
weather.head()

,time,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,windSpeed,cloudCover,uvIndex,visibility,windBearing,location,windGust,pressure,precipType,ozone,precipAccumulation
0,2014-07-15 05:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,10.0,10.00,9.99,1.00,1.02,0.44,0.0,10.003,NaN,bosa,NaN,NaN,NaN,NaN,NaN
1,2014-07-15 06:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,9.0,9.00,9.00,1.00,1.02,0.44,0.0,10.003,50.0,bosa,NaN,NaN,NaN,NaN,NaN
2,2014-07-15 07:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,9.0,9.00,9.00,1.00,1.02,0.44,0.0,10.003,NaN,bosa,NaN,NaN,NaN,NaN,NaN
3,2014-07-15 08:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,10.0,8.46,9.00,0.94,3.09,0.44,0.0,10.003,120.0,bosa,NaN,NaN,NaN,NaN,NaN
4,2014-07-15 09:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,9.0,8.46,6.99,0.87,1.55,0.44,0.0,10.003,270.0,bosa,NaN,NaN,NaN,NaN,NaN


In [87]:
dataset['timestamp_floor'] =  dataset.timestamp.dt.round('H')

In [88]:
weather.time = pd.to_datetime(weather.time)

In [89]:
weather.location.unique()

array(['bosa', 'engativa', 'tunjuelito', 'fontibon', 'kennedy',
       'ciudadbolivar', 'la candelaria', 'puente aranda', 'rafael uribe',
       'sumapaz', 'antonio nariño', 'barrios unidos', 'chapinero',
       'los martires', 'san cristobal', 'santafe', 'suba', 'teusaquillo',
       'usaquen', 'usme'], dtype=object)

In [90]:
weather.replace({"tunjuelito": 'TUNJUELITO', "chapinero":'CHAPINERO', "kennedy": 'KENNEDY', "usaquen": 'USAQUEN', "barrios unidos":  'BARRIOS UNIDOS', "ciudadbolivar": 
'CIUDAD BOLIVAR', "fontibon":  'FONTIBON', "puente aranda":  'PUENTE ARANDA', "suba":  'SUBA', "teusaquillo":
'TEUSAQUILLO', "santafe":  'SANTA FE', "bosa":  'BOSA', "san cristobal":  'SAN CRISTOBAL', "engativa":  'ENGATIVA', "rafael uribe": 
'RAFAEL URIBE URIBE', "antonio nariño":  'ANTONIO NARIÑO', "los martires":  'LOS MARTIRES', "la candelaria":
'CANDELARIA', "usme": 'USME', "sumapaz":  'SUMAPAZ'}, inplace=True)

In [91]:
dataset = dataset.merge(weather, left_on=['borough' , 'timestamp_floor'], right_on=['location', 'time'])

In [92]:
dataset.columns

Index(['x', 'y', 'timestamp', 'date', 'year', 'month', 'day', 'day_of_year',
       'day_of_week', 'hour', 'address', 'severity', 'accident_type',
       'severity_numeric', 'borough', 'cluster_id', 'sample_type',
       'borough_id', 'area_km2', 'population', 'population_density',
       'timestamp_floor', 'time', 'summary', 'icon', 'precipIntensity',
       'precipProbability', 'temperature', 'apparentTemperature', 'dewPoint',
       'humidity', 'windSpeed', 'cloudCover', 'uvIndex', 'visibility',
       'windBearing', 'location', 'windGust', 'pressure', 'precipType',
       'ozone', 'precipAccumulation'],
      dtype='object')

In [93]:
dataset.drop(columns = ['timestamp_floor', 'location'], inplace=True)

In [94]:
dataset.rename(columns={"time": "timestamp_darksky"}, inplace=True)

In [95]:
dataset.to_csv('../data/dataset_2019_with_negatives.csv', index=None)

In [96]:
dataset

,x,y,timestamp,date,year,month,day,day_of_year,day_of_week,hour,...,windSpeed,cloudCover,uvIndex,visibility,windBearing,windGust,pressure,precipType,ozone,precipAccumulation
0,-74.1344,4.5793,2019-01-11 14:20:00,2019-01-11,2019,1,11,11,4,14,...,1.66,0.89,4.0,13.489,167.0,1.66,1018.0,rain,239.8,NaN
1,-74.0488,4.66514,2019-01-11 11:30:00,2019-01-11,2019,1,11,11,4,11,...,1.11,0.83,0.0,12.254,128.0,1.15,1017.2,rain,240.5,NaN
2,-74.1733,4.61559,2019-01-11 23:20:00,2019-01-11,2019,1,11,11,4,23,...,1.65,0.81,0.0,6.814,10.0,2.07,1014.8,rain,237.9,NaN
3,-74.0251,4.72529,2019-01-11 06:30:00,2019-01-11,2019,1,11,11,4,6,...,0.97,0.90,0.0,15.389,41.0,0.99,1017.1,rain,242.7,NaN
4,-74.04462795299997,4.752516783000034,2019-01-11 06:00:00,2019-01-11 00:00:00,2019,1,11,11,4,6,...,0.97,0.90,0.0,15.389,41.0,0.99,1017.1,rain,242.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58128,-74.13568662999995,4.580801026000074,2019-12-03 16:00:00,2019-12-03 00:00:00,2019,12,3,337,1,16,...,2.03,0.64,9.0,15.267,120.0,3.27,1014.6,rain,240.9,NaN
58129,-74.084274684,4.590370525000028,2019-01-12 04:00:00,2019-01-12 00:00:00,2019,1,12,12,5,4,...,1.04,0.93,0.0,16.093,131.0,1.78,1017.5,rain,238.6,NaN
58130,-74.09775169999993,4.619584603000021,2019-10-04 16:00:00,2019-10-04 00:00:00,2019,10,4,277,4,16,...,2.22,0.80,8.0,16.093,146.0,3.88,1016.4,rain,259.2,NaN
58131,-74.17421686799997,4.576242357000068,2019-12-20 19:00:00,2019-12-20 00:00:00,2019,12,20,354,4,19,...,2.17,0.87,6.0,10.840,112.0,2.96,1012.2,rain,232.8,NaN
